In [3]:
from pymongo import MongoClient
import pandas as pd
import requests
import time
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
client = MongoClient("mongodb://localhost:27017/")

In [5]:
db = client.companies

In [6]:
collection_companies = db.clean_companies

In [7]:
query = collection_companies.find()

In [8]:
data = pd.DataFrame(query)

In [9]:
data.head()

,_id,name,category_code,number_of_employees,founded_year,deadpooled_year,description,latitude,longitude,country,state,city,monedas,total_amount_raised,geo
0,5d83a90255488441b3fff55a,Plaxo,web,50,2002.0,None,Contact Management,37.387845,-122.055197,USA,CA,Sunnyvale,Dolares estadounidenses,2.830000e+07,"{'type': 'Point', 'coordinates': [-122.055197,..."
1,5d83a90255488441b3fff55b,Kyte,games_video,40,2006.0,None,Online & Mobile Video Platform,37.788482,-122.409173,USA,CA,San Francisco,Dolares estadounidenses,2.340000e+07,"{'type': 'Point', 'coordinates': [-122.409173,..."
2,5d83a90255488441b3fff55c,Twitter,social,1300,2006.0,None,Real time communication platform,37.776805,-122.416924,USA,CA,San Francisco,Dolares estadounidenses,1.160000e+09,"{'type': 'Point', 'coordinates': [-122.4169244..."
3,5d83a90255488441b3fff55d,eBay,web,15000,1995.0,None,Online Marketplace,37.295005,-121.930035,USA,CA,San Jose,Dolares estadounidenses,6.700000e+06,"{'type': 'Point', 'coordinates': [-121.930035,..."
4,5d83a90255488441b3fff55e,Jajah,mobile,110,2005.0,None,IP Communications Platform,37.423390,-122.089951,USA,CA,Mountain View,Dolares estadounidenses,3.300000e+07,"{'type': 'Point', 'coordinates': [-122.0899512..."


In [10]:
data.shape

(200, 15)

In [11]:
# Cuento el número de empresas a un kilometro de distancia
def getCompaniesNear(geopoint, max_meters=1000):
    return len(list(db.clean_companies.find({
        "geo": {
            "$near": {
               "$geometry": geopoint,
               "$maxDistance": max_meters
             }
       }
    })))

In [12]:
# Creo una nueva columna con el número de empresas que están a un km de distancia
data["num_companies"] = data["geo"].apply(getCompaniesNear)

In [13]:
# Cuento el número de empleados a un kilometro de distancia

def getEmployees(geopoint, max_meters=1000):
    info = list(db.clean_companies.find({
        "geo": {
            "$near": {
               "$geometry": geopoint,
               "$maxDistance": max_meters
             }
       }
    }))
    
    total = 0
    for e in info:
        total += e["number_of_employees"]
    return total

In [17]:
# Creo una nueva columna con el número de empleados totales en un radio de un km de distancia

In [14]:
data["num_employees"] = data["geo"].apply(getEmployees)

In [15]:
# Cuento el total de dinero a un kilometro de distancia

def getMoney(geopoint, max_meters=1000):
    info = list(db.clean_companies.find({
        "geo": {
            "$near": {
                "$geometry": geopoint,
                "$maxDistance": max_meters
            }
        }
    }))
    
    total = 0
    for e in info:
        total += e["total_amount_raised"]
    return total

In [16]:
# Creo una nueva columna
data["total_money"] = data["geo"].apply(getMoney)

In [17]:
data.head(3)

,_id,name,category_code,number_of_employees,founded_year,deadpooled_year,description,latitude,longitude,country,state,city,monedas,total_amount_raised,geo,num_companies,num_employees,total_money
0,5d83a90255488441b3fff55a,Plaxo,web,50,2002.0,None,Contact Management,37.387845,-122.055197,USA,CA,Sunnyvale,Dolares estadounidenses,2.830000e+07,"{'type': 'Point', 'coordinates': [-122.055197,...",1,50,2.830000e+07
1,5d83a90255488441b3fff55b,Kyte,games_video,40,2006.0,None,Online & Mobile Video Platform,37.788482,-122.409173,USA,CA,San Francisco,Dolares estadounidenses,2.340000e+07,"{'type': 'Point', 'coordinates': [-122.409173,...",14,1532,5.451700e+08
2,5d83a90255488441b3fff55c,Twitter,social,1300,2006.0,None,Real time communication platform,37.776805,-122.416924,USA,CA,San Francisco,Dolares estadounidenses,1.160000e+09,"{'type': 'Point', 'coordinates': [-122.4169244...",3,1487,1.233400e+09


In [ ]:
# df.loc[df['Value'].idxmax()]

In [28]:
# Extraigo la oficina que más empresas tecnológicas tiene a su alrededor. La longitud y latitud de esta firma 
# puede servir de referencia para instalar mi empresa. Es del mismo sector.

office = data.loc[data["num_companies"].idxmax()]
office

_id                                             5d83a90255488441b3fff58d
name                                                             Rupture
category_code                                                games_video
number_of_employees                                                   25
founded_year                                                         NaN
deadpooled_year                                                     None
description                                        Gaming Social Network
latitude                                                         37.7839
longitude                                                       -122.395
country                                                              USA
state                                                                 CA
city                                                       San Francisco
monedas                                          Dolares estadounidenses
total_amount_raised                                

In [29]:
data["category_code"].value_counts()

software       70
web            57
games_video    35
mobile         21
social         10
analytics       7
Name: category_code, dtype: int64

In [ ]:
video_games = data

In [22]:
# Exporto el dataframe como un csv para visualizar los datos en Tableau

In [18]:
data.to_csv("./output/data.csv", index=False)

In [ ]:
# Extracción de datos de la Api

In [23]:
zomato_key = os.getenv("zomato_key")

In [22]:
def getVeganRestaurants(lat, lon):
    
    # Hacer una llamada a la api para buscar restaurantes veganos
    # Utilizo un radio de 1 km a la redonda
    # utilizo la clave 308, la cifra que se corresponde con los locales vegetarianos
    
    headers = {
        "user-key": "{}".format(zomato_key)
    }
    url = "https://developers.zomato.com/api/v2.1/search?lat={}&lon={}&radius=1000&cuisines=308".format(lat, lon)
    response = requests.get(url, headers=headers)
    response = response.json()
    return response

In [24]:
restaurants = data.apply(lambda x: getVeganRestaurants(x["latitude"], x["longitude"]), axis=1)

In [59]:
def getInfoRestaurants(data):
    info_restaurants = []
    for rest in data:
        for element in rest["restaurants"]:
            restaurant_dict = {
                   "name": element["restaurant"]["name"],
                    "address": element["restaurant"]["location"]["address"],
                   "lat": element["restaurant"]["location"]["latitude"],
                   "lon": element["restaurant"]["location"]["longitude"],
                   "timings": element["restaurant"]["timings"]
               }
            info_restaurants.append(restaurant_dict)
    return info_restaurants


In [63]:
vegan_restaurants = pd.DataFrame(getInfoRestaurants(restaurants))

In [64]:
vegan_restaurants.head()

,name,address,lat,lon,timings
0,Merit Vegetarian,"548 Lawrence Expressway, Sunnyvale 94085",37.3849611111,-121.9949333333,11 AM to 9 PM (Mon-Sun)
1,Swathi Tiffins,"1202 Apollo Way, Sunnyvale 94085",37.3790150000,-121.9949620000,"10 AM to 10 PM (Mon-Fri),8:30 AM to 10 PM (Sat..."
2,Great Vegi Land,"562 S Murphy Avenue, Sunnyvale 94086",37.3692410000,-122.0324560000,"11 AM to 2:30 PM, 5 PM to 9 PM (Mon-Sun)"
3,Panchavati Kitchen Indian Vegetarian Restaurant,"460 Persian Dr, Sunnyvale 94089",37.4055950000,-122.0133190000,"4 PM to 8 PM (Mon),11 AM to 8 PM (Tue-Sun)"
4,City Kabob,"755 S Mathilda Avenue, Sunnyvale 94087",37.3660320000,-122.0361830000,"11 AM to 7 PM (Mon-Fri),11 AM to 5 PM (Sat-Sun)"


In [65]:
vegan_restaurants.to_csv("./output/vegan_restaurants.csv", index=False)